In [2]:
from mock import patch
import numpy as np
import pandas as pd
from pathlib import Path
from glob import glob

from allensdk.brain_observatory.behavior.behavior_session import BehaviorSession
from allensdk.brain_observatory.behavior.behavior_ophys_experiment import BehaviorOphysExperiment
from allensdk.brain_observatory.behavior.behavior_project_cache.behavior_project_cache import \
    VisualBehaviorOphysProjectCache

C:\Users\iryna.yavorska\Anaconda3\envs\vbo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import allensdk
allensdk.__version__

'2.15.1'

Below are some example data access code for BehaviorSession and OphysExperiment data. For the session/experiment objects themselves, I'm looking for any feedback regarding missing or incorrectly formated data. For the pandas tables the question regards any incorrect typing or formating of the data. If you find issues beyond some that are already know (for instance `age_in_days` being float and not int, mouseid being typed `object`, or the ophys_experiment NWB I produced below missing a project code.) please create an issue on the AllenSDK. Thanks!

The code below requires the `rc/12.15.2` branch of the AllenSDK. To install from the command line (assuming you have `git` installed) use:

`git clone https://github.com/AllenInstitute/AllenSDK`

`cd your_newly_cloned_AllenSDK_directory`

`git checkout ticket/PSB-12/dev`

`pip install -e .`

In [7]:
base_path = Path('//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/vbo_2023_release_candidate')
s3_cache_path = Path('//allen/aibs/informatics/chris.morrison/vbo_nbw_test/allensdk_cache')

In [8]:
cache = VisualBehaviorOphysProjectCache.from_s3_cache(s3_cache_path)

Tables
=====

Below we force the VisualBehaviorOphysProjectCache we loaded from S3 to reload the project metadata but this time using the updated metadata tables for the candidate release. This forces candidate release tables to go through the same code that parses certain columns (e.g. driver_line conversion from string -> list of strings). This should match what the end user will eventually see when downloading the data through the SDK and using the cache to parse it. Make sure the cell below is run when restarting the notebook kernel.

In [9]:
with patch.object(cache.fetch_api,
                  '_get_metadata_path',
                  return_value=base_path / 'project_metadata/behavior_session_table.csv'):
    cache.fetch_api._get_behavior_session_table()
with patch.object(cache.fetch_api,
                  '_get_metadata_path',
                  return_value=base_path / 'project_metadata/ophys_session_table.csv'):
    cache.fetch_api._get_ophys_session_table()
with patch.object(cache.fetch_api,
                  '_get_metadata_path',
                  return_value=base_path / 'project_metadata/ophys_experiment_table.csv'):
    cache.fetch_api._get_ophys_experiment_table()
with patch.object(cache.fetch_api,
                  '_get_metadata_path',
                  return_value=base_path / 'project_metadata/ophys_cells_table.csv'):
    cache.fetch_api._get_ophys_cells_table()

In [11]:
behavior_session_table = cache.get_behavior_session_table()


In [21]:
behavior_session_table.sample(5)

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,session_type,cre_line,indicator,...,correct_reject_trial_count,engaged_trial_count,ophys_session_id,imaging_plane_group_count,ophys_experiment_id,ophys_container_id,targeted_areas,num_dpeths_per_area,date_of_acquisition,project_code
behavior_session_id,,,,,,,,,,,,,,,,,,,,,
885546697,CAM2P.4,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,456564,Ai94(TITL-GCaMP6s),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,112,OPHYS_3_images_A,Slc17a7-IRES2-Cre,GCaMP6s,...,32,346,885401643.0,0.0,[885933191],[902748564],1.0,1.0,2019-06-12 11:15:46.165000+00:00,VisualBehavior
986652189,BEH.B-Box3,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,485688,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,128,TRAINING_5_images_B_epilogue,Sst-IRES-Cre,GCaMP6f,...,25,73,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-19 08:41:38.557000+00:00,VisualBehaviorTask1B
954206178,BEH.B-Box6,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,476970,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,116,TRAINING_1_gratings,Sst-IRES-Cre,GCaMP6f,...,2,207,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-24 13:32:30.706000+00:00,VisualBehaviorTask1B
947306051,CAM2P.5,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,470784,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,140,OPHYS_0_images_B_habituation,Sst-IRES-Cre,GCaMP6f,...,44,0,NaN,NaN,NaN,NaN,NaN,NaN,2019-09-16 11:51:55.072000+00:00,VisualBehaviorTask1B
1074471407,BEH.G-Box2,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,548950,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",M,138,TRAINING_5_images_B_handoff_ready,Slc17a7-IRES2-Cre,GCaMP6f,...,43,328,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-04 09:31:06.618000+00:00,VisualBehaviorTask1B


In [20]:
behavior_session_table.groupby(['equipment_name']).mean()['targeted_areas']

C:\Users\iryna.yavorska\AppData\Local\Temp\ipykernel_10380\1724300209.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  behavior_session_table.groupby(['equipment_name']).mean()['targeted_areas']


equipment_name
BEH.B              NaN
BEH.B-Box1         NaN
BEH.B-Box2         NaN
BEH.B-Box3         NaN
BEH.B-Box4         NaN
BEH.B-Box5         NaN
BEH.B-Box6         NaN
BEH.D              NaN
BEH.D-Box1         NaN
BEH.D-Box2         NaN
BEH.D-Box3         NaN
BEH.D-Box4         NaN
BEH.D-Box5         NaN
BEH.D-Box6         NaN
BEH.F              NaN
BEH.F-Box1         NaN
BEH.F-Box2         NaN
BEH.F-Box3         NaN
BEH.F-Box4         NaN
BEH.F-Box5         NaN
BEH.F-Box6         NaN
BEH.G              NaN
BEH.G-Box1         NaN
BEH.G-Box2         NaN
BEH.G-Box3         NaN
BEH.G-Box4         NaN
BEH.G-Box5         NaN
BEH.G-Box6         NaN
CAM2P.3       1.000000
CAM2P.4       1.000000
CAM2P.5       1.000000
MESO.1        2.774436
Name: targeted_areas, dtype: float64

In [22]:
behavior_session_table.groupby(['equipment_name']).mean()['num_dpeths_per_area']

C:\Users\iryna.yavorska\AppData\Local\Temp\ipykernel_10380\1395709995.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  behavior_session_table.groupby(['equipment_name']).mean()['num_dpeths_per_area']


equipment_name
BEH.B              NaN
BEH.B-Box1         NaN
BEH.B-Box2         NaN
BEH.B-Box3         NaN
BEH.B-Box4         NaN
BEH.B-Box5         NaN
BEH.B-Box6         NaN
BEH.D              NaN
BEH.D-Box1         NaN
BEH.D-Box2         NaN
BEH.D-Box3         NaN
BEH.D-Box4         NaN
BEH.D-Box5         NaN
BEH.D-Box6         NaN
BEH.F              NaN
BEH.F-Box1         NaN
BEH.F-Box2         NaN
BEH.F-Box3         NaN
BEH.F-Box4         NaN
BEH.F-Box5         NaN
BEH.F-Box6         NaN
BEH.G              NaN
BEH.G-Box1         NaN
BEH.G-Box2         NaN
BEH.G-Box3         NaN
BEH.G-Box4         NaN
BEH.G-Box5         NaN
BEH.G-Box6         NaN
CAM2P.3       1.000000
CAM2P.4       1.000000
CAM2P.5       1.000000
MESO.1        3.225564
Name: num_dpeths_per_area, dtype: float64

In [23]:
ophys_session_table = cache.get_ophys_session_table()
ophys_session_table.sample(5)

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,session_type,cre_line,indicator,...,image_set,experience_level,behavior_session_id,imaging_plane_group_count,ophys_experiment_id,ophys_container_id,targeted_areas,num_dpeths_per_area,date_of_acquisition,project_code
ophys_session_id,,,,,,,,,,,,,,,,,,,,,
989281032,CAM2P.3,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,480753,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,156,OPHYS_6_images_A,Sst-IRES-Cre,GCaMP6f,...,images_A,Familiar,989362600,0,[989594023],[978206330],1,1,2019-11-22 09:26:33.567000+00:00,VisualBehaviorTask1B
1081012515,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,546605,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],M,182,OPHYS_2_images_A_passive,Sst-IRES-Cre,GCaMP6f,...,images_A,Familiar,1081053903,4,"[1081264129, 1081264133, 1081264136, 1081264139]","[1079027842, 1079027847, 1079027857, 1079027861]",2,4,2021-02-03 09:49:39.819000+00:00,VisualBehaviorMultiscope
849600749,CAM2P.4,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,436662,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",M,140,OPHYS_3_images_A,Slc17a7-IRES2-Cre,GCaMP6f,...,images_A,Familiar,849648618,0,[850489605],[846213087],1,1,2019-04-11 12:20:52.232000+00:00,VisualBehavior
934576432,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,459773,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,174,OPHYS_1_images_A,Slc17a7-IRES2-Cre,GCaMP6f,...,images_A,Familiar,934812249,0,[935440133],[930024022],1,1,2019-08-29 09:42:53.917000+00:00,VisualBehavior
938359348,CAM2P.3,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,467951,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],M,141,OPHYS_5_images_A_passive,Vip-IRES-Cre,GCaMP6f,...,images_A,Familiar,938745166,0,[939319851],[928325203],1,1,2019-09-04 12:10:32.185000+00:00,VisualBehaviorTask1B


In [24]:
ophys_session_table.groupby(['equipment_name']).mean()['num_dpeths_per_area']

C:\Users\iryna.yavorska\AppData\Local\Temp\ipykernel_10380\3528059636.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ophys_session_table.groupby(['equipment_name']).mean()['num_dpeths_per_area']


equipment_name
CAM2P.3    1.000000
CAM2P.4    1.000000
CAM2P.5    1.000000
MESO.1     3.225564
Name: num_dpeths_per_area, dtype: float64

In [25]:
ophys_experiment_table = cache.get_ophys_experiment_table()
ophys_experiment_table

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,session_type,cre_line,indicator,...,container_workflow_state,experiment_workflow_state,isi_experiment_id,imaging_depth,targeted_structure,published_at,targeted_imaging_depth,date_of_acquisition,project_code,passive
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
951980471,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,150,VISp,2021-03-25,150,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980473,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,225,VISp,2021-03-25,225,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980475,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,75,VISp,2021-03-25,75,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980479,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,150,VISl,2021-03-25,150,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980481,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,225,VISl,2021-03-25,225,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993590480,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,128,OPHYS_2_images_B_passive,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-10 12:25:49.549000+00:00,VisualBehaviorTask1B,True
993862120,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,129,OPHYS_4_images_A,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-11 08:57:44.877000+00:00,VisualBehaviorTask1B,False
994053909,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,130,OPHYS_5_images_A_passive,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-12 09:29:20.785000+00:00,VisualBehaviorTask1B,True


In [26]:
ophys_cells_table = cache.get_ophys_cells_table()
ophys_cells_table

,cell_specimen_id,ophys_experiment_id,x,y,height,width
cell_roi_id,,,,,,
1080884343,1086496928,775614751,63,380,19,17
1080884173,1086496914,775614751,272,463,15,19
1080883843,1086496838,775614751,210,361,17,17
1080886674,1086491756,775614751,136,230,17,15
1080885658,1086491699,775614751,45,435,20,14
...,...,...,...,...,...,...
1115385589,1120127229,1110928073,166,212,19,19
1115385513,1120127083,1110928073,117,68,19,17
1115301580,1120126794,1110928075,419,417,15,17


NWB Files
========

The `VisualBehaviorOphysProjectCache` ends up using the `from_nwb_path` smethod without any extra maniuplationsto load from disk so there's no need to patch over the cache's method. However, I'll put two functions below to make it a bit more convienent to load the NWBs from disk on Linux.

In [27]:
def get_behavior_session(behavior_session_id: int) -> BehaviorSession:
    """Load a candidate BehaviorSession from disk.
    
    Parameters
    ----------
    behavior_session_id : int
        Id Behavior session to load
        
    Returns
    -------
    behavior_session : BehaviorSession
        Loaded behavior session.
    """
    return BehaviorSession.from_nwb_path(
        base_path / f"behavior_sessions/{behavior_session_id}.nwb")

def get_ophys_experiment(ophys_experiment_id: int) -> BehaviorSession:
    """Load a candidate BehaviorOphysExperiment from disk.
    
    Parameters
    ----------
    ophys_experiment_id : int
        Id of ophys experiment to load
        
    Returns
    -------
    ophys_experiment : BehaviorOphysExperiment
        Loaded ophys experiment.
    """
    return BehaviorOphysExperiment.from_nwb_path(
        base_path / f"behavior_ophys_experiments/{ophys_experiment_id}.nwb")

In [28]:
behavior_session = get_behavior_session(870987812)
behavior_session.metadata

{'equipment_name': 'BEH.D-Box1',
 'sex': 'F',
 'age_in_days': 80,
 'stimulus_frame_rate': 60.0,
 'session_type': 'TRAINING_0_gratings_autorewards_15min',
 'date_of_acquisition': datetime.datetime(2019, 5, 17, 14, 46, 33, 550000, tzinfo=tzutc()),
 'reporter_line': 'Ai148(TIT2L-GC6f-ICL-tTA2)',
 'cre_line': 'Sst-IRES-Cre',
 'behavior_session_uuid': UUID('68eaf198-92bb-4548-b5b4-da5ca03a8a68'),
 'driver_line': ['Sst-IRES-Cre'],
 'mouse_id': '457841',
 'project_code': 'VisualBehaviorMultiscope',
 'full_genotype': 'Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt',
 'behavior_session_id': 870987812}

In [32]:
behavior_session = get_behavior_session(843203333)
behavior_session.metadata

{'equipment_name': 'CAM2P.3',
 'sex': 'M',
 'age_in_days': 109,
 'stimulus_frame_rate': 60.0,
 'session_type': 'OPHYS_2_images_A_passive',
 'date_of_acquisition': datetime.datetime(2019, 3, 29, 8, 58, 48, 934000, tzinfo=tzutc()),
 'reporter_line': 'Ai93(TITL-GCaMP6f)',
 'cre_line': 'Slc17a7-IRES2-Cre',
 'behavior_session_uuid': UUID('1e7c6c6c-04d5-4ffb-a888-a848d01c27f5'),
 'driver_line': ['Camk2a-tTA', 'Slc17a7-IRES2-Cre'],
 'mouse_id': '440298',
 'project_code': 'VisualBehavior',
 'full_genotype': 'Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt',
 'behavior_session_id': 843203333}

In [31]:
ophys_experiment_table.loc[843519218]['behavior_session_id']

843203333

In [29]:
ophys_experiment = get_ophys_experiment(843519218)
ophys_experiment.metadata

{'equipment_name': 'CAM2P.3',
 'sex': 'M',
 'age_in_days': 109,
 'stimulus_frame_rate': 60.0,
 'session_type': 'OPHYS_2_images_A_passive',
 'date_of_acquisition': datetime.datetime(2019, 3, 29, 8, 58, 48, 934000, tzinfo=tzutc()),
 'reporter_line': 'Ai93(TITL-GCaMP6f)',
 'cre_line': 'Slc17a7-IRES2-Cre',
 'behavior_session_uuid': UUID('1e7c6c6c-04d5-4ffb-a888-a848d01c27f5'),
 'driver_line': ['Camk2a-tTA', 'Slc17a7-IRES2-Cre'],
 'mouse_id': '440298',
 'project_code': 'VisualBehavior',
 'full_genotype': 'Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt',
 'behavior_session_id': 843203333,
 'indicator': 'GCaMP6f',
 'emission_lambda': 520.0,
 'excitation_lambda': 910.0,
 'ophys_container_id': 843049827,
 'field_of_view_height': 512,
 'field_of_view_width': 452,
 'imaging_depth': 375,
 'targeted_imaging_depth': 375,
 'imaging_plane_group': None,
 'imaging_plane_group_count': 0,
 'ophys_experiment_id': 843519218,
 'ophys_frame_rate': 31.0,
 'ophys_session_id': 843049997,
 'targeted_str

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,session_type,cre_line,indicator,...,container_workflow_state,experiment_workflow_state,isi_experiment_id,imaging_depth,targeted_structure,published_at,targeted_imaging_depth,date_of_acquisition,project_code,passive
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
951980471,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,150,VISp,2021-03-25,150,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980473,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,225,VISp,2021-03-25,225,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980475,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,75,VISp,2021-03-25,75,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980479,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,150,VISl,2021-03-25,150,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
951980481,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206,OPHYS_1_images_A,Sst-IRES-Cre,GCaMP6f,...,published,passed,848974280,225,VISl,2021-03-25,225,2019-09-20 09:59:38.837000+00:00,VisualBehaviorMultiscope,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993590480,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,128,OPHYS_2_images_B_passive,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-10 12:25:49.549000+00:00,VisualBehaviorTask1B,True
993862120,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,129,OPHYS_4_images_A,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-11 08:57:44.877000+00:00,VisualBehaviorTask1B,False
994053909,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,489066,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,130,OPHYS_5_images_A_passive,Vip-IRES-Cre,GCaMP6f,...,published,passed,967103425,175,VISp,2021-03-25,175,2019-12-12 09:29:20.785000+00:00,VisualBehaviorTask1B,True


In [43]:
oeidtotest = 942596349

In [44]:
dataset = BehaviorOphysExperiment.from_lims(oeidtotest)

c:\users\iryna.yavorska\documents\github\allensdk\allensdk\brain_observatory\behavior\data_objects\running_speed\running_processing.py:366: UserWarning: Time array is 1 value shorter than encoder array. Last encoder value removed

  warnings.warn(
c:\users\iryna.yavorska\documents\github\allensdk\allensdk\brain_observatory\behavior\data_objects\running_speed\running_processing.py:366: UserWarning: Time array is 1 value shorter than encoder array. Last encoder value removed

  warnings.warn(
c:\users\iryna.yavorska\documents\github\allensdk\allensdk\brain_observatory\behavior\data_objects\running_speed\running_processing.py:366: UserWarning: Time array is 1 value shorter than encoder array. Last encoder value removed

  warnings.warn(
Reading image_names from pkl file: 100%|███████████████████████████████████████████████| 76/76 [00:57<00:00,  1.32it/s]
c:\users\iryna.yavorska\documents\github\allensdk\allensdk\brain_observatory\behavior\data_objects\eye_tracking\rig_geometry.py:233: Use

In [42]:
dataset.metadata

{'equipment_name': 'MESO.1',
 'sex': 'F',
 'age_in_days': 206,
 'stimulus_frame_rate': 60.0,
 'session_type': 'OPHYS_1_images_A',
 'date_of_acquisition': Timestamp('2019-09-20 09:59:38.837000+0000', tz='UTC'),
 'reporter_line': 'Ai148(TIT2L-GC6f-ICL-tTA2)',
 'cre_line': 'Sst-IRES-Cre',
 'behavior_session_uuid': UUID('592420fb-911d-41c5-8763-4ebdeb0ef416'),
 'driver_line': ['Sst-IRES-Cre'],
 'mouse_id': '457841',
 'project_code': 'VisualBehaviorMultiscope',
 'full_genotype': 'Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt',
 'behavior_session_id': 951520319,
 'indicator': 'GCaMP6f',
 'emission_lambda': 520.0,
 'excitation_lambda': 910.0,
 'ophys_container_id': 1018028342,
 'field_of_view_height': 512,
 'field_of_view_width': 512,
 'imaging_depth': 150,
 'targeted_imaging_depth': 150,
 'imaging_plane_group': 0,
 'imaging_plane_group_count': 4,
 'ophys_experiment_id': 951980471,
 'ophys_frame_rate': 11.0,
 'ophys_session_id': 951410079,
 'targeted_structure': 'VISp'}

### Checking lims and stim file differences in mouse ids

In [47]:
behavior_session_table = cache.get_behavior_session_table()

In [58]:
behavior_session_table[behavior_session_table.mouse_id == '453988'][['date_of_acquisition']].loc[929341697]#453911

date_of_acquisition   2019-08-22 09:05:57.330000+00:00
Name: 929341697, dtype: datetime64[ns, UTC]

In [64]:
behavior_session_table[behavior_session_table.mouse_id == '453988'][['date_of_acquisition', 'session_type']]

,date_of_acquisition,session_type
behavior_session_id,,
858098096,2019-04-26 11:17:42.498000+00:00,TRAINING_0_gratings_autorewards_15min
859064345,2019-04-29 14:02:46.682000+00:00,TRAINING_1_gratings
859735301,2019-04-30 12:45:15.762000+00:00,TRAINING_1_gratings
860650174,2019-05-01 11:39:00.517000+00:00,TRAINING_1_gratings
861785387,2019-05-02 13:33:51.108000+00:00,TRAINING_1_gratings
...,...,...
934610593,2019-08-29 09:12:54.578000+00:00,OPHYS_5_images_B_passive
935811563,2019-08-30 10:01:30.475000+00:00,OPHYS_6_images_B
937416292,2019-09-03 08:59:14.478000+00:00,OPHYS_3_images_A


In [62]:
behavior_session_table[behavior_session_table.mouse_id == '453911'][['date_of_acquisition', 'session_type']]

,date_of_acquisition,session_type
behavior_session_id,,
858104885,2019-04-26 11:23:07.408000+00:00,TRAINING_0_gratings_autorewards_15min
858900546,2019-04-29 09:00:05.293000+00:00,TRAINING_1_gratings
859601488,2019-04-30 08:52:48.185000+00:00,TRAINING_1_gratings
860247669,2019-05-01 08:39:08.379000+00:00,TRAINING_1_gratings
861717498,2019-05-02 09:27:25.521000+00:00,TRAINING_1_gratings
...,...,...
916403851,2019-08-02 08:53:07.844000+00:00,OPHYS_6_images_B
918103715,2019-08-06 09:02:21.903000+00:00,OPHYS_6_images_B
923410363,2019-08-14 08:58:56.375000+00:00,OPHYS_1_images_A
